In [7]:
import pandas as pd
import time
from sentence_transformers import SentenceTransformer, util
df = pd.read_csv('./AES/processed_data.csv', encoding = 'ISO-8859-1')

df1 = df[['essay']]

df_dict = df1.to_dict('dict')

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

embedding_1= model.encode(df_dict['essay'][0], convert_to_tensor=True)


meas_dict = {}
essayCheckNo=0
meas_dict[essayCheckNo]=1
meas_dict

start_time = time.time()
for key in df_dict['essay']:
    if key != 0:
        embedding_2 = model.encode(df_dict['essay'][key], convert_to_tensor=True)
        meas = util.pytorch_cos_sim(embedding_1, embedding_2)
        meas_dict[key]=meas.item()

end_time = time.time()
elapsed_time =(end_time-start_time)
print(elapsed_time)

402.60151171684265


In [57]:
# create a new dataframe composed of cosine similarity
df_meas_dict = pd.DataFrame(meas_dict,index=[0]).transpose()
df_meas_dict.rename({0: 'cossim'}, axis='columns',inplace=True)
df_meas_dict['cossim'][0]=1
df_meas_dict

,cossim
0,1.000000
1,0.663650
2,0.606666
3,0.565070
4,0.610151
...,...
12971,0.162224
12972,0.079628
12973,0.088495
12974,0.073543


In [68]:
# create a new dataframe composed of wordcounts
df_count = pd.DataFrame(df1['essay'].str.split().apply(len))
df_count.rename({'essay': 'wordcount'}, axis='columns',inplace=True)
df_count

,wordcount
0,333
1,409
2,272
3,485
4,461
...,...
12971,793
12972,514
12973,764
12974,543


In [77]:
df_filterable = pd.concat([df1,df_meas_dict,df_count],axis=1)
df_filterable.head()

,essay,cossim,wordcount
0,"Dear local newspaper, I think effects computer...",1.000000,333
1,Dear I believe that using computers will benef...,0.663650,409
2,"Dear, More and more people use computers, but ...",0.606666,272
3,"Dear Local Newspaper, I have found that many e...",0.565070,485
4,Dear I know having computers has a positive ef...,0.610151,461


In [86]:
# get the top whatever number of essays
threshold = 0.69

df_highcosmatch = pd.DataFrame(df_filterable['cossim'] > threshold)
df_highcosmatch.rename({'cossim': 'highcosmatch'}, axis='columns',inplace=True)
df_highcosmatch.value_counts()

highcosmatch
False           12864
True              112
dtype: int64

In [87]:
df_filterable = pd.concat([df_filterable,df_highcosmatch],axis=1)
df_filterable.head()

,essay,cossim,wordcount,highcosmatch
0,"Dear local newspaper, I think effects computer...",1.000000,333,True
1,Dear I believe that using computers will benef...,0.663650,409,False
2,"Dear, More and more people use computers, but ...",0.606666,272,False
3,"Dear Local Newspaper, I have found that many e...",0.565070,485,False
4,Dear I know having computers has a positive ef...,0.610151,461,False


In [88]:
df_filterable.query('highcosmatch')

,essay,cossim,wordcount,highcosmatch
0,"Dear local newspaper, I think effects computer...",1.000000,333,True
14,"Dear, In this world today we should have every...",0.706164,175,True
33,Dear is a rumor going around saying that compu...,0.712626,357,True
37,"Dear local Newspaper, in the society we live i...",0.735292,352,True
55,"Over the past decade, the percentage of people...",0.709133,466,True
...,...,...,...,...
1719,Dear NewsPaper Have you ever taken into consid...,0.714009,439,True
1745,"Dear Local Newspaper, People all around the gl...",0.731787,294,True
1766,"Computers, a topic if you ask me. Sure they ar...",0.706272,361,True
1771,"Technology, such as computers are very big. I ...",0.704179,384,True


In [100]:
df_filterbyhighcos = df_filterable.query('highcosmatch')

In [119]:
df_filterbyhighcos.to_csv('Filtered/highcos.csv',index=False)

In [ ]:
from io import StringIO

csvStr = df_filterbyhighcos.to_csv(index=False)

df_new = pd.read_csv(StringIO(csvStr))